### 🧠 What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [1]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

In [2]:
# Step 1: Load and embed the document
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

e:\RAGUdemy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FFD9A0B380>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FFD97D4050>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [5]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question=decomposition_chain.invoke({"question": query})


In [6]:
print(decomposition_question)

Here's a breakdown of the complex question into smaller sub-questions:

1. **What types of memory mechanisms does LangChain offer for its models?**  (This focuses on LangChain's specific memory capabilities)
2. **How do LangChain agents utilize memory in their decision-making processes?** (This delves into the practical application of memory within LangChain agents)
3. **What memory and retrieval strategies does CrewAI employ for its AI agents?** (This shifts the focus to CrewAI's approach to memory)
4. **How do the memory and agent functionalities of LangChain and CrewAI differ in terms of implementation and capabilities?** (This encourages a comparative analysis) 


These sub-questions target different aspects of the original query, allowing for more focused and precise document retrieval. 



In [7]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [8]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]
    
    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input": subq, "context": docs})
        results.append(f"Q: {subq}\nA: {result}")
    
    return "\n\n".join(results)

In [9]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: Here are some sub-questions to break down the complex question:
A: Please provide the complex question! I need to know what question you're trying to break down before I can provide sub-questions. 

For example, you could ask:

* "How does CrewAI work?"
* "What are the advantages of using LangChain agents?"
* "Can you give me an example of a multi-step workflow that could be improved by CrewAI?"


Once you give me the complex question, I can help you break it down into more manageable sub-questions.  



Q: **What types of memory mechanisms does LangChain employ?** (This focuses on LangChain's specific memory capabilities)
A: LangChain uses memory modules such as **ConversationBufferMemory** and **ConversationSummaryMemory**. 


These modules allow the LLM to remember past conversation turns and summarize long interactions. 


Q: **How do LangChain agents utilize memory?** (This explores the application of memory within LangChain's agent framework)
A: LangChain agen